In [ ]:
#   This program details the approach to solving for the optimal reimbursements (among other things) for the insurers in the simultaneous bargaining case with linear demand and no recapture. The way things are defined here allows for both active or passive beliefs. This is a test document to work through the approach associated with taking this simulation approach, which uses covariance_beliefs.ipynb as a reference point. 

In [2]:
import numpy as np
from scipy.optimize import minimize,fsolve
import matplotlib.pyplot as plt
import pandas as pd

ImportError: DLL load failed: The specified module could not be found.

In [32]:
#   Computing the price $p_i$ offered by each insurer $i$. Since this is the linear demand case, we already have these functional forms, but in the logit case, this likely needs additional details. As calculated in the paper in closed form, these assume interior solutions.
#   In contrast to covariance_beliefs.ipynb, I use "lam" instead of "cost" to convey lambda (without the lambda command arising) and "v" instead of "wtp" to convey value v for the consumers. 
def calc_price(phi1, phi2, lam, v):
    p1 = (2 * phi1 + phi2) / 3. + lam
    p2 = (2 * phi2 + phi1) / 3. + lam
    return p1, p2
#   Computing the market share $s_i$ captured by each insurer $i$. We begin by assuming exogenous recapture, which we no longer think is the case. (Namely, this portion implements Equation (9) from the current draft, which assumes the Hotelling solution for market share.) I'll pursue endogenous recapture after successfully replicating this code. 
def calc_s(phi1, phi2, lam, v):
    p1, p2 = calc_price(phi1, phi2, lam, v)
    t_hat = .5 + (p2 - p1) / (2. * lam)
    return t_hat
#   Computing the profits for each insurer. Since we are looking at the simultaneous case, insurers are symmetric, which simplifies items. "mc" corresponds to the cost associated with the hospital of processing claims associated with each insurer. In this case, mc is assumed to be the same for both insurers. 
def calc_profits(phi1, phi2, lam, v, mc):
    #   This is the market share detailed in the previous line. Market share is calculated for insurer 1, with insurer 2 assumed to capture the remainder in this case, where all consumers buy insurance. 
    s = calc_s(phi1, phi2, lam, v)
    #   These are the prices detailed previously. 
    p1, p2 = calc_price(phi1, phi2, lam, v)
    #   These are the profits for each insurer. The comma allows us to define both in-line.
    profits1, profits2 = s * (p1 - phi1), (1-s) * (p2 - phi2)
    #   This is the hospital profit. By assumption, the market is fully captured by the two insurers.
    mc1, mc2 = mc
    hosp_profit = s * (phi1 - mc1) + (1-s) * (phi2 - mc2)
    return hosp_profit, profits1, profits2
#   Computing the disagreement payoff $\pi_h(\mathcal{G}\setminus i)$. Note that we have assumed that $v$ is sufficiently high, which we may need to relax later. 
def disagreement_simult(phi1, phi2, lam, v, mc, active = False):
    #   Cost of each insurer to the hospital is the same. 
    mc1, mc2 = mc
    #   Without recapture:
    s_hat = calc_s(phi1, phi2, lam, v)
    p1, p2 = calc_price(phi1, phi2, lam, v)
    #   With recapture, we have the following. "np.clip" limits the values in an array, namely bounding the share between 0 and 1. This encapsulates the idea of recapture in that this is the market share captured by the first insurer, so long as the consumers have tastes which are uniformly distributed: the consumers willing to buy from insurer 2 are such that $v-p_2-(1-\lambda)t_i>0$ which is true iff $t
    s_hat = np.clip((lam + p2 - v) / lam, 0, 1) 
    if active: #    I assume this is equivalent to 'if active = True:'
        return (v - lam) / 2 - mc2  #   With active beliefs, the disagreement payoff of the hospital is such that the second hospital captures the entire market with the monopolist reimbursement, $\frac{v-lam}{2}$. 
    else:   # does there need to be else here, or does simply having return afterwards account for the fact that it isn't active? (active = False?)
        return (1 - s_hat) * (phi2 - mc2) # If not active beliefs (passive beliefs case), then the hospital's payoff is equivalent to the market share captured by insurer 2 (assuming full recapture)
#   Defining the Nash-in-Nash objective function.
def nash_in_nash(phi1, phi2, lam, v, mc, beta = .5, active = False):
    hosp_profit, profits1, profits2 = calc_profits(phi1, phi2, lam, v, mc) # These are each defined in the previous section defining calc_profits, and calculates and recalls each to this "section."
    #   With the objective function itself, we negate it in order to minimize rather than maximize. In the following case, we simplified by taking the log of the expression as a whole and separating it out that way. I also change "active = active" to "active = False", since I'm not entirely sure what that does in this case. Finally, I'm not really following why the "max" operator only applies to hosp_profit - disagreement_simult. 
    obj = -1 * (np.log(max(hosp_profit - disagreement_simult(phi1, phi2, lam, v, mc, active = False), 1e-4)) * (1 - beta) + np.log(profits1) * beta)
    # I'm including a case where I don't take logs, and applied the max operator to the entire expression. 
    # obj = -1 * (max(((hosp_profit - disagreement_simult(phi1, phi2, lam, v, mc, active = False)) ** (1 - beta)) * ((profits1) ** (beta)), 1e-4))
    return obj
#   A test case with \lambda = 5, v = 10, and c = 0.
LAM = 5
V = 10
MC = np.array([0,0]) #  zero marginal cost example
print(nash_in_nash(1.5 * LAM, 1.5 * LAM, LAM, V, MC))   # This takes phi1 and phi2 as given. Rather than 2 * LAM as in the current code, I think this is intended to be 1.5, since (3/2)\lambda is the solution for phi in this case.  

-1.4655968762082099


In [53]:
#   The previous section took phi1 and phi2 as given. We want to solve each firm's choice problem for their optimal reimbursement $\phi_i$ while holding the other reimbursement $\phi_j$ fixed. 
def bargain_helper(phi1, phi2, lam, v, mc, beta = .5, active = False):
    result = minimize(nash_in_nash, phi1, args = (phi2, lam, v, mc, beta, active), method = 'Nelder-Mead', options = {'disp' : False})
    return result.x #   .x returns the final answer, since 'result' is a class of items.
#   We then have an example to check whether the $\phi_1$ arrived at by the program is consistent with our results thus far. 
LAM = 5
V = 10
MC = np.array([0,0])
print(bargain_helper(15, 15, LAM, V, MC, beta = .5, active = False)) #  Is this not still taking phi1 and phi2 as given at 15? I'm not really following this. In this case, we should get $\phi=\frac{3}{2}\lambda$, but this certainly isn't what we arrive at (this generates 8.35501099 instead of 7.5). Where do the portions for 15 come from? Even with phi1=phi2=7.5 I still don't get the right answer here. Further, with my changes to the previous section under nash_in_nash, nothing changed. 
#   This mistake doesn't stem from the fact that this is recalling nash_in_nash, which is negated, since we negated the max in order to apply the minimize feature with respect to phi1 here. 

[8.35501099]
